# **Sydney vs. Melbourne**


---


##**Introduction**
### For a long time, there has always been a heated debate about which Australian city is better, Sydney  or Melbourne. So much so that the capital of this continent country is **NOT** either of the two, but is located **between** them (see the map) as a compromise and probably to avoid civil unrest. 


<br>

---

<br>

##**Business Problem** <br>
### I'm not brave enough to settle this dispute, but this project aims to give the reader an idea of the Sydney and Melbourne's landscape. It can be useful to people who are at at the fork of making a decision to move to the biggest cities in Australia depending on their current need and interest. So which city is for you?

## 1.) Import libraries

In [25]:
#for data analysis
import numpy as np
import pandas as pd

#for getting geo data
from geopy.geocoders import Nominatim

#for machine learning  - clustering
from sklearn.cluster import KMeans

#for data visualization

import matplotlib.cm as cm
import matplotlib.colors as colors

#for webscraping and handling requests
import requests
from bs4 import BeautifulSoup

from pandas.io.json import json_normalize

#filtering results of webscraping
import re

#map rendering
import folium

2.)Webscraping the suburb names of Melbourne

In [26]:
#get request
melb_url = 'https://en.m.wikipedia.org/wiki/List_of_Melbourne_suburbs'
melb_results = requests.get(melb_url).content

#Parse the html data to beautifulsoup object
soup = BeautifulSoup(melb_results, 'html.parser')

#Initialize an empty list of the suburbs
melb_suburbs = []

#Append the suburb data from the BeautifulSoup object to the list


In [27]:
#the html contents are not formatted in a table; the suburbs are contained in bullets <li>
suburb_raw = []
for i in range(1,5):
  for row in soup.find('div', class_='mw-parser-output').find('section', class_='mf-section-{} collapsible-block'.format(i)).find_all('ul'):
    for j in row.find_all('li'):
       suburb_raw.append(j.get_text())

#using regex to only choose the ones with postal code in it. D means not digit, limited to 35 letters to exclude sentences.  
#Parentheses return the a tuple of 2 items (surburb (\D{0,35}), and postal(\d+)
suburb_pattern = re.compile(r'^(\D{0,35})(\d+)')



#after a lot of trial and error , loop, regex, pandas combo worked
melb_data =pd.DataFrame(columns=['Suburb', 'Postal Code', 'Query'])
for item in suburb_raw:
  item = suburb_pattern.findall(item)
  if item != []:
    query = ' '.join(item[0])
    suburb = item[0][0]
    postal = item[0][1]
    melb_data = melb_data.append({'Suburb': suburb, 'Postal Code': postal, 'Query': query}, ignore_index= True)

In [28]:
melb_data.head()

,Suburb,Postal Code,Query
0,Carlton,3053,Carlton 3053
1,Carlton North,3054,Carlton North 3054
2,Docklands,3008,Docklands 3008
3,East Melbourne,3002,East Melbourne 3002
4,Flemington,3031,Flemington 3031


In [29]:
#find the suburbs that contain parenthesis, then replace them in the 
exclude = re.compile('(\D*)(\(\D*\))  (\d+)')
exclude_list = []
exclude_post = []
exclude_suburb = []

for i in melb_data.Query:
  item = exclude.findall(i)
  if item != []:
    exclude_list.append(item)
    exclude_post.append(item[0][2])
    exclude_suburb.append(item[0][0])
print(exclude_list)
print(exclude_post)
print(exclude_suburb )


[[('Malvern ', '(/ˈmɔːlvən/)', '3144')], [('Prahran ', "(/pɛ'ræn/)", '3181')]]
['3144', '3181']
['Malvern ', 'Prahran ']


In [30]:
melb_data[melb_data['Postal Code'].isin(exclude_post)]

,Suburb,Postal Code,Query
483,Kooyong,3144,Kooyong 3144
484,Malvern (/ˈmɔːlvən/),3144,Malvern (/ˈmɔːlvən/) 3144
486,Prahran (/pɛ'ræn/),3181,Prahran (/pɛ'ræn/) 3181
489,Windsor,3181,Windsor 3181


In [31]:
melb_data['Query'].replace(to_replace = ['Malvern (/ˈmɔːlvən/)  3144', 'Prahran (/pɛ\'ræn/)  3181'], value = ['Malvern 3144', 'Prahran 3181'], inplace = True)

In [32]:
melb_data.iloc[484:487,] #to check if the query is renamed. it will be used to get the coordinates for geocoder

,Suburb,Postal Code,Query
484,Malvern (/ˈmɔːlvən/),3144,Malvern 3144
485,Malvern East,3145,Malvern East 3145
486,Prahran (/pɛ'ræn/),3181,Prahran 3181


In [33]:
#@title Trial to get coordinates
#Geopy got a bit unreliable. one time it worked perfectly, the next, it timed out. so proceed to plan B
'''geolocator = Nominatim(user_agent="ML_explorer")

melb_data['Query'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

melb_data['Coord']= melb_data['Query'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

# Split the coordinates into Lat and Long, drop the M
melb_data[['Latitude', 'Longitude']] =melb_data['Coord'].apply(pd.Series)
'''

'geolocator = Nominatim(user_agent="ML_explorer")\n\nmelb_data[\'Query\'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))\n\nmelb_data[\'Coord\']= melb_data[\'Query\'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))\n\n# Split the coordinates into Lat and Long, drop the M\nmelb_data[[\'Latitude\', \'Longitude\']] =melb_data[\'Coord\'].apply(pd.Series)\n'

In [34]:
!pip install geocoder
import geocoder

In [35]:

def get_latlng(suburb):
    # initialize your lat_lng_coords to None; making a function will help handle the timed out error
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Melbourne, Australia'.format(suburb))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(suburb) for suburb in melb_data["Suburb"].tolist()]


In [36]:
# Create temporary dataframe to populate the coordinates into Latitude and Longitude
temp = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the original dataframe
melb_data['Latitude'] = temp['Latitude']
melb_data['Longitude'] = temp['Longitude']
melb_data.head()

,Suburb,Postal Code,Query,Latitude,Longitude
0,Carlton,3053,Carlton 3053,-37.80101,144.96951
1,Carlton North,3054,Carlton North 3054,-37.78918,144.97188
2,Docklands,3008,Docklands 3008,-37.81926,144.94555
3,East Melbourne,3002,East Melbourne 3002,-37.81132,144.97799
4,Flemington,3031,Flemington 3031,-37.78814,144.92965


In [89]:
melb_merged = melb_data.groupby('Postal Code')['Suburb'].apply(lambda s: ",".join(s)).to_frame().reset_index()
melb_merged

,Postal Code,Suburb
0,3000,Melbourne
1,3002,East Melbourne
2,3003,West Melbourne
3,3004,"Melbourne ,Melbourne"
4,3006,"Southbank ,South Wharf ,Southbank ,South Wharf"
...,...,...
241,3978,"Cardinia ,Clyde ,Clyde North"
242,3980,"Blind Bight ,Tooradin ,Warneet"
243,3981,"Bayles ,Catani ,Dalmore ,Heath Hill ,Koo Wee Rup"
244,3984,"Caldermeade ,Lang Lang ,Monomeith"


# PART 3
## **DATA ANALYSIS**

Data Analysis Melbourne

In [37]:
address_MEL = "Melbourne, VIC"
location_mel = geocoder.arcgis(address_MEL)
latitude_mel = location_mel.latlng[0]
longitude_mel = location_mel.latlng[1]
print('Melbourne\'s coordinates are : Latitude ( {} ) , Longitude ( {} )'.format(latitude_mel, longitude_mel))


Melbourne's coordinates are : Latitude ( -37.81738999999993 ) , Longitude ( 144.96751000000006 )


In [40]:
map_melbourne = folium.Map(location = [latitude_mel, longitude_mel], zoom_start= 10)

for lat, lng, combined in zip(melb_data['Latitude'],melb_data['Longitude'], melb_data['Query']):
  label = folium.Popup(combined, parse_html=True)
  folium.CircleMarker(
      location = [lat,lng],
      radius = 5,
      popup = label,
      color = 'blue',
      fill = True,
      fill_color = '#14B6B0',
      fill_opacity = 0.6,
      parse_html = False).add_to(map_melbourne)

map_melbourne

In [92]:
melb_merged.head()

,Postal Code,Suburb
0,3000,Melbourne
1,3002,East Melbourne
2,3003,West Melbourne
3,3004,"Melbourne ,Melbourne"
4,3006,"Southbank ,South Wharf ,Southbank ,South Wharf"


# **3. Explore using Foursquare API**

In [41]:
#@title CREDENTIALS
#credentials
CLIENT_ID = 'EA1V0LWDFSZ13O0GV0S5ZRBCP1MKEC150VKEAXWWZPTOJMCV' 
CLIENT_SECRET = '5OM21UMNWI5NF4OC42FLHH3HITXMMWBOJWIRHP34OUWDJNEA'
VERSION = '20210101'
LIMIT = 100

Explore neighborhood

In [ ]:
'''url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&limit={}&ll={},{},&radius={}'.format(CLIENT_ID, CLIENT_SECRET,VERSION,LIMIT,latitude_mel,longitude_mel,500)
  #in that order (CLIENT_ID, CLIENT_SECRET,VERSION,LIMIT,latitude_mel,longitude_mel,500)
results = requests.get(url).json()
results'''

In [ ]:
'''melb_venues = getNearbyVenues(suburbs = melb_data['Suburb'], latitudes= melb_data['Latitude'], longitudes = melb_data['Longitude'], radius= 500)
melb_venues'''

In [90]:
def getNearbyvenues(names, latitudes, longitudes, radius = 500):
  venues_list= [] #initialize an empty list
  for name, lat, lng in zip(names, latitudes, longitudes): #for loop to take the data of interest then appending them in the initialized list, venues_list
    #print(name)

    #create the API request url
    url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&LIMIT={}&ll={},{}&radius={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        LIMIT,
        lat,
        lng,
        radius
    )

    #make the request
    results=  requests.get(url).json()['response']['groups'][0]['items']

    #return only the relevant information for each nearby venues
    venues_list.append([(
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results #list comprehesion to take out values from results
        ])
  nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list]) #list comprehension; unpack items in venue list to package in a dataframe
  nearby_venues.columns = ['Neighborhood',
                            'Neighborhood Latitude',
                            'Neighborhood Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category'
                            
                            ] #renaming column names
  
  return(nearby_venues)

In [91]:
melb_venues = getNearbyvenues(names = melb_merged['Suburb'], latitudes = melb_merged['Latitude'], longitudes = melb_merged['Longitude'], radius = 500)

KeyError: ignored

In [44]:
melb_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Carlton,-37.80101,144.96951,D.O.C. Pizza & Mozzarella Bar,-37.798954,144.968490,Pizza Place
1,Carlton,-37.80101,144.96951,Carlton Wine Room,-37.798584,144.968610,Wine Bar
2,Carlton,-37.80101,144.96951,Gewürzhaus,-37.799050,144.967480,Gourmet Shop
3,Carlton,-37.80101,144.96951,Assembly Coffee & Tea,-37.802750,144.967290,Coffee Shop
4,Carlton,-37.80101,144.96951,Baker D. Chirico,-37.798788,144.968499,Bakery
...,...,...,...,...,...,...,...
4290,Windsor,-37.85616,144.99257,The Woods,-37.854316,144.993124,Restaurant
4291,Windsor,-37.85616,144.99257,Windsor Castle Hotel,-37.855745,144.987911,Pub
4292,Windsor,-37.85616,144.99257,Tusk Cafe Bar,-37.853304,144.992947,Café
4293,Windsor,-37.85616,144.99257,The Thai,-37.855211,144.992931,Thai Restaurant


In [46]:
#Exploratory Data Analysis

print ('There are {} unique categories'.format(len(melb_venues['Venue Category'])))

There are 4295 unique categories


In [48]:
#Analyzing each neighborhood

melb_onehot = pd.get_dummies(melb_venues[['Venue Category']], prefix = "", prefix_sep="")

In [49]:
#add neighborhood column back to onehot dataframe
melb_onehot['Suburb'] = melb_data['Suburb']

#put the Suburb column to first
suburb_col = melb_onehot.pop('Suburb')
melb_onehot.insert(0, 'Suburb', suburb_col)

#have a quick look at the columns
melb_onehot.head()


,Suburb,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Boat Launch,Bookstore,Boutique,Bowling Alley,Bowling Green,Boxing Gym,...,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Swim School,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Tour Provider,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Xinjiang Restaurant,Yoga Studio
0,Carlton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Carlton North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,Docklands,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,East Melbourne,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Flemington,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
#group by suburb then get mean of frequency

melb_grouped = melb_onehot.groupby('Suburb').mean().reset_index()
melb_grouped.tail()

,Suburb,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Boat Launch,Bookstore,Boutique,Bowling Alley,Bowling Green,Boxing Gym,...,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Swim School,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Tour Provider,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Xinjiang Restaurant,Yoga Studio
442,Yarra Junction,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
443,Yarrambat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
444,Yellingbo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
445,Yering,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446,Yuroke,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [53]:
#confirm size
melb_grouped.shape

(447, 315)

In [58]:
num_top_venues = 5
for sub in melb_grouped['Suburb']:
  #print suburb on top
  print('**' + sub + '**')

  #select the corresponding suburb then transpose data to categories line up vertically
  temp = melb_grouped[melb_grouped['Suburb'] == sub].T.reset_index()
  
  temp.columns = ['Venues', 'Frequency']

  #Neighborhood is the row, the succeeding ones are Venues and Frequency
  temp = temp.iloc[1:]
  temp['Frequency'] = temp['Frequency'].astype(float)
  temp = temp.round({'Frequency': 2})

  #sort the values of the transposed data by frequency in descendin order then print the top venues
  print(temp.sort_values('Frequency', ascending=False).reset_index(drop = True).head(num_top_venues))
  print ('\n')

**Abbotsford **
                 Venues  Frequency
0    Italian Restaurant        1.0
1        Adult Boutique        0.0
2                Office        0.0
3           Outlet Mall        0.0
4  Outdoor Supply Store        0.0


**Albert Park **
                 Venues  Frequency
0    Italian Restaurant        1.0
1        Adult Boutique        0.0
2                Office        0.0
3           Outlet Mall        0.0
4  Outdoor Supply Store        0.0


**Alphington **
                  Venues  Frequency
0                   Café        0.5
1  Vietnamese Restaurant        0.5
2         Adult Boutique        0.0
3                 Office        0.0
4   Outdoor Supply Store        0.0


**Armadale **
                 Venues  Frequency
0              Tea Room        1.0
1        Adult Boutique        0.0
2         Movie Theater        0.0
3  Outdoor Supply Store        0.0
4     Outdoor Sculpture        0.0


**Arthurs Seat **
                 Venues  Frequency
0           Music Venue       